In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()
train_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
x_test=test_data
x_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
y_train=train_data['Survived'].to_numpy()
train_data.drop(['Survived'], axis=1, inplace=True)

In [5]:
x_train=train_data
x_train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
from pandas.api.types import is_numeric_dtype
def get_categorical_indicies(x_train):
    cats = []
    for col in x_train.columns:
        if is_numeric_dtype(x_train[col]):
            pass
        else:
            cats.append(col)
    cat_indicies = []
    for col in cats:
        cat_indicies.append(x_train.columns.get_loc(col))
    return cat_indicies
categorical_indicies = get_categorical_indicies(x_train)

def convert_cats(x_train):
    cats = []
    for col in x_train.columns:
        if is_numeric_dtype(x_train[col]):
            pass
        else:
            cats.append(col)
    cat_indicies = []
    for col in cats:
        x_train[col] = x_train[col].astype('category')
convert_cats(x_train)

In [7]:
from pandas.api.types import is_numeric_dtype
def get_categorical_indicies(x_test):
    cats = []
    for col in x_test.columns:
        if is_numeric_dtype(x_test[col]):
            pass
        else:
            cats.append(col)
    cat_indicies = []
    for col in cats:
        cat_indicies.append(x_test.columns.get_loc(col))
    return cat_indicies
categorical_indicies1 = get_categorical_indicies(x_test)

def convert_cats(x_test):
    cats = []
    for col in x_test.columns:
        if is_numeric_dtype(x_test[col]):
            pass
        else:
            cats.append(col)
    cat_indicies = []
    for col in cats:
        x_test[col] = x_test[col].astype('category')
convert_cats(x_test)

In [8]:
print(categorical_indicies)
print(categorical_indicies1)

[2, 3, 7, 9, 10]
[2, 3, 7, 9, 10]


In [9]:
zero_data_set_sum=x_train.isnull().sum()
zero_data_set_percentage=x_train.isnull().sum()/x_train.shape[0]*100
zero_data_set=pd.DataFrame({'sum':zero_data_set_sum, 'percentage': zero_data_set_percentage})
zero=zero_data_set[zero_data_set['sum']>0].head(40)

In [10]:
zero

,sum,percentage
Age,177,19.865320
Cabin,687,77.104377
Embarked,2,0.224467


In [11]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='constant', fill_value=-999)
column_to_impute=[]
for i in range(len(zero.index)):
    column_to_impute.append(zero.index[i])
for col in column_to_impute:
    imputed_column = imputer.fit_transform(x_train[[col]])
    x_train[col] = imputed_column
for col in column_to_impute:
    imputed_column = imputer.fit_transform(x_test[[col]])
    x_test[col] = imputed_column
    

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [12]:
for col,name in enumerate(x_train):
    if col in categorical_indicies:
        pass
    else:
        x_train[name] = x_train[name].astype(str)
for col,name in enumerate(x_test):
    if col in categorical_indicies:
        pass
    else:
        x_test[name] = x_test[name].astype(str)

In [13]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    object  
 1   Pclass       891 non-null    object  
 2   Name         891 non-null    category
 3   Sex          891 non-null    category
 4   Age          891 non-null    object  
 5   SibSp        891 non-null    object  
 6   Parch        891 non-null    object  
 7   Ticket       891 non-null    category
 8   Fare         891 non-null    object  
 9   Cabin        891 non-null    object  
 10  Embarked     891 non-null    object  
dtypes: category(3), object(8)
memory usage: 121.0+ KB


In [14]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(891, 11)
(418, 11)
(891,)


In [15]:
from sklearn.model_selection import train_test_split
x_train2, x_test2, y_train2, y_test2 = train_test_split(x_train, y_train, test_size=0.1, random_state=2)

In [16]:
import catboost as cb
train_dataset = cb.Pool(x_train2,y_train2,
                        cat_features=categorical_indicies)
test_dataset=cb.Pool(x_test2, y_test2, cat_features=categorical_indicies)

In [17]:
from catboost import CatBoostClassifier, Pool
CAT=cb.CatBoostClassifier(iterations=10000,
                          metric_period=100,
#                           eval_metric='Accuracy',
                          learning_rate=0.02,
                          early_stopping_rounds = 50,
                          depth=11,
                         )
CAT.fit(train_dataset,
        eval_set=test_dataset
       )

0:	learn: 0.6818241	test: 0.6828561	best: 0.6828561 (0)	total: 59ms	remaining: 9m 50s
100:	learn: 0.3308110	test: 0.4384676	best: 0.4384676 (100)	total: 1.43s	remaining: 2m 20s
200:	learn: 0.2541757	test: 0.4262327	best: 0.4243963 (193)	total: 3.05s	remaining: 2m 28s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.4243963241
bestIteration = 193

Shrink model to first 194 iterations.


In [18]:
import catboost as cb
train_dataset = cb.Pool(x_train,y_train,
                        cat_features=categorical_indicies)

In [19]:
CAT=cb.CatBoostClassifier(iterations=200,
#                           metric_period=100,
#                           eval_metric='Accuracy',
                          learning_rate=0.02,
                          early_stopping_rounds = 50,
                          depth=11,
                         )
CAT.fit(train_dataset
       )

0:	learn: 0.6795237	total: 28.8ms	remaining: 5.72s
1:	learn: 0.6697939	total: 31.8ms	remaining: 3.15s
2:	learn: 0.6582622	total: 37.3ms	remaining: 2.45s
3:	learn: 0.6480543	total: 41.7ms	remaining: 2.04s
4:	learn: 0.6365184	total: 64.2ms	remaining: 2.5s
5:	learn: 0.6306162	total: 66.2ms	remaining: 2.14s
6:	learn: 0.6186813	total: 87.6ms	remaining: 2.42s
7:	learn: 0.6102999	total: 91.9ms	remaining: 2.21s
8:	learn: 0.6030368	total: 94.2ms	remaining: 2s
9:	learn: 0.5939222	total: 99.6ms	remaining: 1.89s
10:	learn: 0.5867468	total: 105ms	remaining: 1.8s
11:	learn: 0.5767395	total: 124ms	remaining: 1.95s
12:	learn: 0.5684875	total: 143ms	remaining: 2.06s
13:	learn: 0.5614301	total: 156ms	remaining: 2.07s
14:	learn: 0.5523438	total: 183ms	remaining: 2.26s
15:	learn: 0.5441196	total: 202ms	remaining: 2.32s
16:	learn: 0.5378775	total: 205ms	remaining: 2.2s
17:	learn: 0.5340486	total: 208ms	remaining: 2.11s
18:	learn: 0.5270123	total: 228ms	remaining: 2.17s
19:	learn: 0.5227583	total: 231ms	rem

In [20]:
predictions=CAT.predict(x_test)
predictions

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [21]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
